Setting up matplotlib and using a nicer set of plot parameters

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import sep
from astropy.visualization import astropy_mpl_style
plt.style.use(astropy_mpl_style)
import astropy.io
from astropy.io import fits

rcParams['figure.figsize'] = [10.,8.]

reading an image from a fits file

In [ ]:
#read image into standard 2-d numpy array
data = fits.read("../data/image.fits")

showing the image

In [ ]:
m , s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();

## background subtraction

measuring a spatially varying background on our image

In [ ]:
bkg = sep.Background(data)

get a global mean and noise of image background

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

evaluate background as 2-d array, same size as original image

In [ ]:
bkg_image = np.array(bkg)

show the background

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

evaluate background as 2-d array, same size as original image again

In [ ]:
bkg_rms = bkg.rms()

show background noise

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

subtract the background from the data array

In [ ]:
bkg.subfrom(data)

## now run object detection on background-subtracted data

setting detection threshold

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

using sep.extract because it has many options for controlling detection threshold, pixel masking, filtering, and object deblending.

objects is a numpy structured array with many fields

In [ ]:
#how many objects were detected?
len(objects)

objects['x'], and objects['y'] will give the centroid coordinates of the objects. Just to check where the detected objects are, we’ll over-plot the object coordinates with some basic shape parameters on the image:

In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

objects has many other fields, giving information such as second moments, and peak pixel positions and values

In [ ]:
# available fields
objects.dtype.names

## aperature photometry
performing a simple cirular aperature photometry with a 3 pixel radius at the locations of the objects using flux, fluxerr, and flag. these are all 1-d arrays with one entry per object

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err=bkg.globalrms, gain=1.0)

showing the first 10 objects results

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))